<a href="https://colab.research.google.com/github/joshbrx/DataScience_challenge/blob/main/Departamentos_en_la_CDMX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Pregunta 1***
El archivo ``reto_precios.csv`` contiene información sobre precios de departamentos nuevos dentro de la CDMX, junto con diversas características del inmueble.

Para esta pregunta,requerimos que nos puedas explicar los factores que influyen en el precio por metro cuadrado de cada vivienda

***Hint: Existe información externa que te podría ser útil para resolver esta pregunta*.**



---



In [1]:
import pandas as pd
import numpy as np
import keras
import re

In [2]:
url = "https://raw.githubusercontent.com/joshbrx/DataScience_challenge/main/reto_precios.csv"
df = pd.read_csv(url)
df.head()

,main_name,subtitle,link,location,price,attributes,timestamp,id,address,since,...,apartments_per_floor,disposition,parking_lots,floor_situated,orientation,num_bedrooms,department_type,m2,final_price,price_square_meter
0,Departamento - Roma Norte,Departamento en venta,https://departamento.metroscubicos.com/MLM-139...,"Sinaloa 20 , Roma Sur, Roma Sur, Cuauhtémoc, D...",9000000 pesos,"127 m² construidos,2 recs.",2022-03-29,1,"Sinaloa 20 , Roma Sur",Publicado hace 5 días,...,NaN,Contrafrente,2,NaN,NaN,2.0,NaN,127.0,9000000.0,70866.141732
1,Preventa Exclusiva · Terraloza Roma Norte ·,Departamento en venta,https://departamento.metroscubicos.com/MLM-139...,"Tonala, Roma Norte, Cuauhtémoc, Distrito Federal",4264000 pesos,"54 m² construidos,1 rec.",2022-03-29,2,Tonala,Publicado hace 7 días,...,NaN,NaN,1,NaN,NaN,1.0,NaN,54.0,4264000.0,78962.962963
2,Townhouse Único En La Roma Sur,Departamento en venta,https://departamento.metroscubicos.com/MLM-139...,"Tepeji 43, Roma Sur, Ciudad De México, Cdmx, M...",8093500 pesos,"163 m² construidos,3 recs.",2022-03-29,3,"Tepeji 43, Roma Sur, Ciudad De México, Cdmx, M...",Publicado hace 4 días,...,NaN,NaN,2,NaN,NaN,3.0,NaN,163.0,8093500.0,49653.374233
3,"Departamento En Roma Sur, Cuauhtémoc Con 2 Rec...",Departamento en venta,https://departamento.metroscubicos.com/MLM-132...,"Linares, Roma Sur, Cuauhtémoc, Distrito Federal",4000000 pesos,"93 m² construidos,2 recs.",2022-03-29,4,Linares,Publicado hace 5 meses por {action},...,NaN,NaN,1,NaN,Oeste,2.0,Loft,93.0,4000000.0,43010.752688
4,Últimos Deptos En Preventa En Roma Norte,Departamento en venta,https://departamento.metroscubicos.com/MLM-137...,"Frontera, Roma Norte, Cuauhtémoc, Distrito Fed...",6732700 pesos,"103 m² construidos,2 recs.",2022-03-29,5,Frontera,Publicado hace 52 días,...,NaN,NaN,1,NaN,NaN,2.0,NaN,103.0,6732700.0,65366.019417


## Análisis exploratorio y limpieza en base de datos

Es necesario poder comprender el tipo de datos con el que estamos trabajando para poder realizar posteriormente una limpieza de los mismos.

In [3]:
# Se define una copia de "df" llamada "data_cdmx" para poder limpiarla y no modificar la base de datos inicial, ni perder información.
data_cdmx = df.copy()

In [4]:
df.info()
# df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   main_name             981 non-null    object 
 1   subtitle              981 non-null    object 
 2   link                  981 non-null    object 
 3   location              981 non-null    object 
 4   price                 981 non-null    object 
 5   attributes            981 non-null    object 
 6   timestamp             981 non-null    object 
 7   id                    981 non-null    int64  
 8   address               702 non-null    object 
 9   since                 981 non-null    object 
 10  description           981 non-null    object 
 11  vendor                981 non-null    object 
 12  lat                   981 non-null    float64
 13  lon                   981 non-null    float64
 14  price_mod             981 non-null    float64
 15  price_currency        9

### Columnas repetidas y triviales

En el primer caso, repiten los mismos valores por lo que no son relevantes

In [5]:
df[["subtitle", "timestamp", "price_currency", "age_in_years", "age_in_years"]].head(10)
data_cdmx = data_cdmx.drop(["subtitle", "timestamp", "price_currency", "age_in_years", "age_in_years"], axis=1);

En el segundo caso, las siguientes columnas son triviales u otorgan información redundante.

* `attributes` (Consta de 2 variables, la primera corresponde con `m2` y la segunda con `num_bedrooms`)

In [6]:
# Hay 2 variables comprimidas en `attributes` (la primera corresponde con la columna `m2` y la segunda con `num_bedrooms`)
df.iloc[:,[0,5,30,32]].head()

,main_name,attributes,num_bedrooms,m2
0,Departamento - Roma Norte,"127 m² construidos,2 recs.",2.0,127.0
1,Preventa Exclusiva · Terraloza Roma Norte ·,"54 m² construidos,1 rec.",1.0,54.0
2,Townhouse Único En La Roma Sur,"163 m² construidos,3 recs.",3.0,163.0
3,"Departamento En Roma Sur, Cuauhtémoc Con 2 Rec...","93 m² construidos,2 recs.",2.0,93.0
4,Últimos Deptos En Preventa En Roma Norte,"103 m² construidos,2 recs.",2.0,103.0


In [7]:
# Los datos de "attributes" y "m2" solo difieren en una pequeña parte
df[df['attributes'].apply(lambda x: float(re.findall(r'\d+', x)[0])) != df['m2']].iloc[:,[0,5,32]].head()

,main_name,attributes,m2
189,"Preventa De Departamentos, Locales Y Oficinas ...","103 m² construidos,3 recs.",103.10
200,Venta Hermoso Depto Col. Roma Sur Baja California,"95 m² construidos,2 recs.",95.20
205,Departamentos Colonia Roma,"53 m² construidos,1 rec.",53.02
218,Departamento En Venta Colonia Roma Sur,"120 m² construidos,2 recs.",120.24
281,"Departamento En Venta Estilo Ecléctico, Roma N...","96 m² construidos,2 recs.",96.50


In [8]:
# Inlcuso los datos de "attributes" no llegan a tener el dato del número de habitaciones por lo que ésta columna no es relevante.
def tmp_func1(x):
    if len(re.findall(r'\d+', x))==2:
        s = re.findall(r'\d+', x)[1]
    else: s = "prueba"
    return s
df[df['attributes'].apply(tmp_func1) == "prueba"].iloc[:,[0,5,30]].head()

,main_name,attributes,num_bedrooms
411,Venta Departamento Nuevo Colonia Roma,141 m² construidos,2.0
541,Departamento En Venta Roma Sur,"7,210 m² construidos,2 recs.",2.0
584,Preventa En La Colonia Roma,80 m² construidos,2.0
601,Preventa De Departamentos En Roma Sur,64 m² construidos,2.0
619,"Preventa En La Colonia Roma Norte, Chapultepec",34 m² construidos,2.0


Dado que se repiten los datos de la columna `attributes` e incluso se encuentran mejor descritos en las columnas `m2` y `num_bedrooms`, se elimina esta columna.

In [9]:
data_cdmx = data_cdmx.drop("attributes", axis=1);

* ``orientation`` (irrelevante y mejor mostrada en `lat` y `lon`)

Dado que solo consta de 8 datos registrados en comparación con 981 de la base de datos, se puede descartar.

In [10]:
print(df.iloc[:,29].value_counts())
data_cdmx = data_cdmx.drop("orientation", axis=1);

Poniente    3
Sur         2
Oeste       1
Norte       1
Oriente     1
Name: orientation, dtype: int64


### Extracción de datos

Se suelen minar los datos relevantes (normalmente valores númericos) de una cadena de texto sin limpiar.

#### `price` (se recupera el valor en pesos en forma numérica)

In [11]:
# df[["id", "price"]].head()
data_cdmx["price"] = df['price'].apply(lambda x: int(re.search(r'\d+', x).group()))
data_cdmx[["id", "price"]].head()

,id,price
0,1,9000000
1,2,4264000
2,3,8093500
3,4,4000000
4,5,6732700


#### ``since`` contra `since_period`, ``days_on_site`` y ``since_value``
Como se puede observar, se puede calcular el valor de dias en el sitio a partir de la columna `since` que luego se subdivide en dos columnas más, sin embargo esto se puede mejorar.

In [12]:
df.iloc[:, [7, 9, 16, 17, 18]].head()

,id,since,since_period,since_value,days_on_site
0,1,Publicado hace 5 días,days,5,5.0
1,2,Publicado hace 7 días,days,7,7.0
2,3,Publicado hace 4 días,days,4,4.0
3,4,Publicado hace 5 meses por {action},months,5,150.0
4,5,Publicado hace 52 días,days,52,52.0


In [13]:
# Refinamos el calculo de los días totales para medirlo en función de un mes y año promedio
def tmp_func2(x):
    tmp_var = int(re.findall('\d+', x)[0])
    if re.findall("meses", x) == ['meses']:
        tmp_var *= 30.42
    if re.findall("años", x) == ["años"]:
        tmp_var *= 365.24
    return int(round(tmp_var))

data_cdmx['since'] = df['since'].apply(tmp_func2)
data_cdmx[["id", "since", "days_on_site"]].head()

,id,since,days_on_site
0,1,5,5.0
1,2,7,7.0
2,3,4,4.0
3,4,152,150.0
4,5,52,52.0


Una vez mostrada la comparación se puede continuar a eliminar las columnas adicionales

In [25]:
# Remplazamos estos valores en "days_on_site" y eliminamos las columnas extra
data_cdmx['days_on_site'] = df['since']
data_cdmx = data_cdmx.drop(["since", "since_period", "since_value"], axis=1);

### Completar valores nulos

Se rellenan los valores nulos en las columnas según el contexto o la descripción de la base de datos

* `amenities` (áreas comunes, vigilancia, servicios extra, etc.)



In [15]:
# Falta el caso para cuando no hay amenidades extra, rellenamos valores con 0
# df["amenities"].unique()
data_cdmx["amenities"] = df["amenities"].fillna(0)
data_cdmx["amenities"].unique()

array([3., 2., 0., 4., 5., 6., 7., 8., 1.])

* ``cellars`` (Almacenes, bovedas, depósitos, etc.)

In [16]:
# Falta el caso para cuando no tienen almacenes/bodegas, completamos valores con 0
# df["cellars"].unique()
data_cdmx["cellars"] = df["cellars"].fillna(0)
data_cdmx["cellars"].unique()

array([0., 2., 1.])

* ``monthly_fee`` (Gastos de mantenimiento, servicios, entre otros)

In [17]:
# Falta el caso no hay gastos de mantenimiento, completamos valores con 0 MXN y limpiamos los datos
# df["monthly_fee"].unique()
data_cdmx["monthly_fee"] = df["monthly_fee"].fillna("0 MXN")
data_cdmx['monthly_fee'] = data_cdmx['monthly_fee'].apply(lambda x: int(re.search(r'\d+', x).group()))
data_cdmx['monthly_fee'].unique()

array([   0,  853,  800,    1, 3700, 2000, 1200, 2500,    4, 4000, 1500,
       2900, 3900, 3000])

* `apartments_per_floor` (nùmero de apartamentos por piso)

In [18]:
# Falta el caso para cuando no hay informacion así que rellenamos valores con 1, sabiendo que mínimo hay un departamento por piso
df["apartments_per_floor"].unique()
data_cdmx["apartments_per_floor"] = df["apartments_per_floor"].fillna(1)
data_cdmx["apartments_per_floor"].unique()

array([1., 2., 7., 3., 4., 5.])

* `disposition` (Distribución del departamentamento en el edificio)

In [19]:
# Falta el caso en que no se proporciona información, rellenamos valores con N/A
# df["disposition"].unique()
data_cdmx["disposition"] = df["disposition"].fillna("N/A")
data_cdmx["disposition"].unique()

array(['Contrafrente', 'N/A', 'Frente', 'Interno'], dtype=object)

* ``floor_situated`` (Piso del edificio en el que se localiza el departamento)

In [20]:
# Falta el caso en el que el edificio solo es de un piso o si el departamento está en planta baja, rellenamos valores con 0
# df["floor_situated"].unique()
data_cdmx["floor_situated"] = df["floor_situated"].fillna(0)
data_cdmx["floor_situated"].unique()

array([0., 1., 3., 2., 4., 5.])

* `department_type` (Loft, penthouse, estudio, duplex)

In [21]:
# Falta tan varios casos por lo que podemos definir una categoría general, rellenamos valores con `Normal` dado que no poseemos más información
# df["department_type"].unique()
data_cdmx["department_type"] = df["department_type"].fillna("Normal")
data_cdmx["department_type"].unique()

array(['Normal', 'Loft'], dtype=object)

### Dato mal capturado en ``bathrooms``

Según la base de datos existe un departamento de $91\ m^2$ con 23 baños, algo que no tiene sentido por lo que opta por eliminar el renglón

In [22]:
df[(df.iloc[:,21] == 23)].iloc[:, [0, 7,21, 32]]

,main_name,id,bathrooms,m2
687,Departamento En Venta Roma Sur,692,23.0,91.0


In [23]:
# Eliminamos este dato en la variable `data_cdmx`
data_cdmx = data_cdmx.drop(687)
data_cdmx["bathrooms"].value_counts()

2.0    686
1.0    178
3.0    105
4.0     10
5.0      1
Name: bathrooms, dtype: int64

## iwiiiii

In [26]:
data_cdmx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 980 entries, 0 to 980
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   main_name             980 non-null    object 
 1   link                  980 non-null    object 
 2   location              980 non-null    object 
 3   price                 980 non-null    int64  
 4   id                    980 non-null    int64  
 5   address               701 non-null    object 
 6   description           980 non-null    object 
 7   vendor                980 non-null    object 
 8   lat                   980 non-null    float64
 9   lon                   980 non-null    float64
 10  price_mod             980 non-null    float64
 11  days_on_site          980 non-null    object 
 12  amenities             980 non-null    float64
 13  bathrooms             980 non-null    float64
 14  cellars               980 non-null    float64
 15  num_floors            8